# Ant-Colony-Optimization Problem

This Jupyter Notebook deals with Ant-Colony-Optimization. All information is taken from Daniel Merkle's and Martin Middendorf's Chapter 14 on Swarm Intelligence in Search Methodologies. Introductory Tutorials in Optimization and Decision Support Techniques by Edmund K. Burke and Graham Kendall.

Ant-Colony Optimization is one of two main areas of Swarm Intelligence. It is used to solve optimization and search problems. The idea is that ant colonies find the shortest way form their nest to the food source with the help of pheromones. Experiments have shown that if you connect the nest with two different branches to a food source - one way is longer than the other one, ants will quickly find out which way is the shortest one. The result is an ant trail.

How exactly does it work? At the beginning both branches are taken with the same probability. After a while the pheromone concentration will be higher at the shorter branch since the ants are back at the nest way faster if they have taken the short road. The pheromone concentration gets higher more easily and since ants are taking the road with a higher pheromone concentration, eventually all ants take the short road

In [81]:
import pandas as pd
import numpy as np

## 1. Initialization
Importing .txt files and saving them as arrays.

In [82]:
def initialize(filepath): 
    with open(filepath, 'r') as t:
        distance_matrix = []
        for line in t.readlines():
            y = [int(value) for value in line.split()]
            distance_matrix.append(y)
        
        t.close()
        distance_matrix = np.array(distance_matrix)
    
    return distance_matrix

## 2. Solution Construction

In [83]:
def solution_construction(distance_matrix, pheromone_matrix, number_ants, alpha, beta):
    
    # test values
    #pheromone_matrix = [[0,0.4,0.1,1.2,0.7],[0.8,0,0.1,1.5,1.7],[0.9,0.5,0,0.2,0.7],[0.1,0.8,1.1,0,0.7],[0.5,0.4,0.1,1.2,0],[0,0.4,0.1,1.2,0.7]]
    #number_ants = 10
    #distance_matrix = [[0,1,5,2,6],[1,0,3,7,4],[5,3,0,3,5],[2,7,3,0,1],[6,4,5,1,0]]
    #alpha = 0.5
    #beta = 0.5

    ant_number = 0
    solution_matrix = []
    len_distance_matrix = len(distance_matrix)
    range_distance_matrix = range(len(distance_matrix))
    
    # creating a solution matrix corresponding to the size of the distance_matrix
    for i in range(number_ants):
        solution_matrix.append([])
        for j in range_distance_matrix:
            solution_matrix[i].append([])
    
    # do for each ant
    while ant_number < number_ants:
        print('solution_construction')
        
        # initialize selection_set, append a placeholder '1' for each city available
        selection_set = []            
        selection_set = [[1] for i in range_distance_matrix]
        
        # choose a random starting city, save it in the solution matrix and set it to '0' in selection_set
        start = (np.random.random_integers(0, len_distance_matrix-1))
        solution_matrix[ant_number][0] = start
        selection_set[start] = 0
        
        # iterating through all cities left
        for x in range(len_distance_matrix-1):
            probability = []                
                
            # create probability matrix of size of distance_matrix    
            for i in range_distance_matrix:
                probability.append([])
                
                # checking if there is still a city left
                if((selection_set[i])!=0):
                    # calculating heuristics
                    variable = solution_matrix[ant_number][x]
                    
                    if alpha == 1 and beta == 1:
                        t= (pheromone_matrix[variable][i])*alpha
                        n = (1/(distance_matrix[variable][i]))*beta
                        sum = 0
                        
                        for j in range_distance_matrix:
                            if((selection_set[j])!=0):
                                sum += ((pheromone_matrix[solution_matrix[ant_number][x]][j])*alpha) * ((1/distance_matrix[solution_matrix[ant_number][x]][j])*beta)
                        
                    elif alpha == 1 and beta == 0:
                        t = (pheromone_matrix[variable][i])*alpha
                        n = (1/(distance_matrix[variable][i]))
                        
                        for j in range_distance_matrix:
                            if((selection_set[j])!=0):
                                sum = ((pheromone_matrix[solution_matrix[ant_number][x]][j])*alpha) * ((1/distance_matrix[solution_matrix[ant_number][x]][j]))
                        
                    else:
                        t = (pheromone_matrix[variable][i])**alpha
                        n = (1/(distance_matrix[variable][i]))**beta
                        
                        for j in range_distance_matrix:
                            if((selection_set[j])!=0):
                                sum += ((pheromone_matrix[solution_matrix[ant_number][x]][j])**alpha) * ((1/distance_matrix[solution_matrix[ant_number][x]][j])**beta)
                    
                    probability[i] = (t*n)/sum
                else:
                    probability[i]= 0
                    

            boundaries = []
            boundaries.append(0)
            boundaries[0] = probability[0]
 
            # calculation of boundaries for roulette-wheel-like selection of next city
            for i in range(1, len(probability)):
                boundaries.append(0)
                boundaries[i] = probability[i]+boundaries[i-1]
            
            # selection of the next city
            rand_mate = np.random.uniform(0,1)
            for m in range(len(boundaries)):
                if boundaries[m] > rand_mate:
                    city = m
                    break
                    
            solution_matrix[ant_number][x+1] = city
            selection_set[m] = 0
            
        ant_number += 1
        
    return solution_matrix

## 3. Evaporation

In [84]:
def evaporate(pheromone_matrix,evaporator):
    pheromone_matrix = pheromone_matrix * evaporator

    return pheromone_matrix

## 4. Intensification

solution_matrix will be a list of lists which each holding the numbers of the cities in the order that  each of the ants visited them


In [85]:
array2=[[1, 1, 1],[1, 1, 1],[1, 1, 1]]
array3=[[1, 2, 0],[0, 2, 1]]
array0=[[0, 4, 1],[1, 0, 5],[4, 1, 0]]

def intensification(distance_matrix, pheromone_matrix, solution_matrix,intensifier):
      
    best_solution, shortest_distance = get_best_solution(distance_matrix, solution_matrix)
   
    for k in range(len(best_solution)):  
        x = best_solution[k]
        if k < len(best_solution)-1: 
            pheromone_matrix[x][best_solution[k+1]]= pheromone_matrix[x][best_solution[k+1]]*intensifier
        else:
            pheromone_matrix[best_solution[k]][0]= pheromone_matrix[best_solution[k]][0]*intensifier
   
    return pheromone_matrix
            
print(intensification(array0,array2,array3,1.05))

[[1, 1, 1.05], [1.05, 1, 1], [1, 1.05, 1]]


In [86]:
def get_best_solution(distance_matrix, solution_matrix):
    shortest_distance = 10000000000
    for i in solution_matrix:
        distance = 0
       
       
        for j in range(len(i)):
            y=i[j]
            if j < len(i)-1: 
                distance = distance + distance_matrix[y][i[j+1]]
            else:
                distance= distance + distance_matrix[i[j]][i[0]]
         
        
        if distance < shortest_distance:
            shortest_distance = distance
            best_solution = i
            #print(best_solution)
    return best_solution, shortest_distance

## Action

In [88]:
# Choose the city matrix: '01_tsp.txt', '02_tsp.txt', '03_tsp.txt'
filepath = '01_tsp.txt'
distance_matrix = initialize(filepath)

# Set up pheromone matrix with shape 150 x 150 and fixed values = 0.01
pheromone_matrix = np.full(distance_matrix.shape, 0.01)

# Set number of ants
number_ants = 10

# Set alpha and beta (test: alpha = 1, beta = 0 AND alpha = 1, beta = 0)
alpha = 1
beta = 0

intensifier = 1.05
evaporator = 0.1


best_solution_list = []
# "Measure" convergence
converged = False
count = 0
totalCount = 0

while not converged: 
    # solution_paths store the order how each ant visited the cities (each row is one ant)
    solution_matrix = solution_construction(distance_matrix, pheromone_matrix, number_ants, alpha, beta)
    print("solution_contruction done!")
    
    # Set up new pheromone matrix
    pheromone_matrix = evaporate(pheromone_matrix,evaporator)

    # Intensify good solutions and store them in a new pheromone matrix
    pheromone_matrix = intensification(distance_matrix, pheromone_matrix, solution_matrix,intensifier)
        
    # Convergence-Criterium:
    best_solution, shortest_distance = get_best_solution(distance_matrix, solution_matrix)
    best_solution_list.append(shortest_distance)
    if count == 10:
    # if best_solution did not change, the algorithm converged, otherwise we reset the count value
        if best_solution_first == shortest_distance:
            converged = True
        else:
            count = 0
    # store the first best_solution for convergence-comparison
    elif count == 0:
        best_solution_first = shortest_distance
        count += 1
    else:
    # increase count
        count += 1
    totalCount += 1
    print("totalCount",totalCount)
    print("best_solution",shortest_distance)
    print("best_solution_list",best_solution_list)    

solution_construction 0


C:\Anaconda\lib\site-packages\ipykernel_launcher.py:28: DeprecationWarning: This function is deprecated. Please call randint(0, 149 + 1) instead


solution_construction 1
solution_construction 2
solution_construction 3
solution_construction 4
solution_construction 5
solution_construction 6
solution_construction 7
solution_construction 8
solution_construction 9
solution_contruction done!
totalCount 1
best_solution 26316
best_solution_list [26316]
solution_construction 0
solution_construction 1
solution_construction 2
solution_construction 3
solution_construction 4
solution_construction 5
solution_construction 6
solution_construction 7
solution_construction 8
solution_construction 9
solution_contruction done!
totalCount 2
best_solution 26406
best_solution_list [26316, 26406]
solution_construction 0
solution_construction 1
solution_construction 2
solution_construction 3
solution_construction 4
solution_construction 5
solution_construction 6
solution_construction 7
solution_construction 8
solution_construction 9
solution_contruction done!
totalCount 3
best_solution 27028
best_solution_list [26316, 26406, 27028]
solution_construction 

KeyboardInterrupt: 